# Time series classification with sktime

Early time series classification (eTSC) is the problem of classifying a time series after as few measurements as possible with the highest possible accuracy. The most critical issue of any eTSC method is to decide when enough data of a time series has been seen to take a decision: Waiting for more data points usually makes the classification problem easier but delays the time in which a classification is made; in contrast, earlier classification has to cope with less input data, often leading to inferior accuracy.

This notebook gives a quick guide to get you started

# Data sets and problem types
The UCR/UEA [time series classification archive](https://timeseriesclassification.com/) contains a large number of example TSC problems that have been used thousands of times in the literature to assess TSC algorithms. These dataset have certain characteristics that influence what data structure we use to store them in memory.

In [26]:
# Imports used in this notebook
import matplotlib

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import warnings

from sktime.classification.early_classification._teaser import TEASER
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datasets import (  # load_basic_motions,; load_japanese_vowels,; load_plaid,
    load_arrow_head,
)

warnings.simplefilter("ignore")

In [27]:
# Load all arrow head
arrow_X, arrow_y = load_arrow_head(return_type="numpy2d")

# Load default train/test splits from sktime/datasets/data
arrow_train_X, arrow_train_y = load_arrow_head(split="train", return_type="numpy2d")
arrow_test_X, arrow_test_y = load_arrow_head(split="test", return_type="numpyflat")

# Building the TEASER Classifier

TEASER is a two-tier model using a slave and a master classifier. As a first tier, TEASER requires a TSC, such as WEASEL, which produces class probabilities as output. As a second tier is an anomaly detector, such as a one-class SVM.

In [28]:
teaser = TEASER(
    random_state=0,
    classification_points=[6, 10, 16, 24],
    estimator=TimeSeriesForestClassifier(n_estimators=10, random_state=0),
    return_safety_decisions=False,
)
teaser.fit(arrow_train_X, arrow_train_y)

TEASER(classification_points=[6, 10, 16, 24],
       estimator=TimeSeriesForestClassifier(n_estimators=10, random_state=0),
       random_state=0, return_safety_decisions=False)

# Determine the accuracy and earliness on the test data

In [29]:
teaser.score(arrow_test_X, arrow_test_y)

0.52